# Multilabel classification

We develop here a model for multilabel classification.

In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

from NegativeClassOptimization import ml

/home/eugen/miniconda3/envs/nco/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = ml.MultilabelSN10(10)
X = torch.zeros((1, 220))
model.forward(X)

tensor([[0.4318, 0.4919, 0.5623, 0.5438, 0.4484, 0.4389, 0.5243, 0.5541, 0.4674,
         0.4987]], grad_fn=<SigmoidBackward0>)

In [3]:
y = torch.tensor(
    [[0.4504, 0.5505, 0.4989, 0.4607, 0.5415, 0.4269, 0.4880, 0.4147, 0.4800, 0.5470]]
)

loss = nn.BCELoss()

ml.compute_loss(
    model, 
    loss, 
    X, 
    y,
)

tensor(0.6978, grad_fn=<BinaryCrossEntropyBackward0>)